In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ln -s /content/gdrive/MyDrive/ /mydrive

In [ ]:
!git clone https://github.com/wprzem/clothes_classifier.git

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet

In [ ]:
!mkdir train

In [ ]:
!make GPU=1 OPENCV=1 CUDNN=1

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74 -O train/darknet53.conv.74

In [ ]:
!cp cfg/yolov3.cfg train/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' train/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' train/yolov3_training.cfg
!sed -i 's/classes=80/classes=8/' train/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 16000/' train/yolov3_training.cfg
!sed -i 's/steps=400000,450000/steps=15500,16500/' train/yolov3_training.cfg
!sed -i '603s/filters=255/filters=39/' train/yolov3_training.cfg
!sed -i '689s/filters=255/filters=39/' train/yolov3_training.cfg
!sed -i '776s/filters=255/filters=39/' train/yolov3_training.cfg

In [ ]:
!mkdir train/obj
!cp ../clothes_classifier/images/* train/obj/

In [ ]:
import glob
import os

images_list = glob.glob('../clothes_classifier/images/*jpg')
train_images_ratio = 0.67
num_train_images = int(len(images_list) * train_images_ratio)
train_images = images_list[:num_train_images]
test_images = images_list[num_train_images:]

with open(os.path.join('train', 'train.txt'), 'w') as f:
  f.write('\n'.join(train_images))

with open(os.path.join('train', 'test.txt'), 'w') as f:
  f.write('\n'.join(test_images))

In [ ]:
!cp ../clothes_classifier/clothes.names train/obj.names

In [ ]:
!echo classes=8 > train/obj.data
!echo train=train/train.txt >> train/obj.data
!echo valid=train/test.txt >> train/obj.data
!echo names=train/obj.names >> train/obj.data
!echo backup=/mydrive/testtraining >> train/obj.data

In [ ]:
!./darknet detector train train/obj.data train/yolov3_training.cfg train/darknet53.conv.74 -dont_show